In [ ]:
#sentiment API call

from google.cloud import language_v1
from google.cloud.language_v1 import enums
import csv

def read_csv(file_name):
    header_added = False
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        next(reader)
    # Reading row by row
        for row in reader:
            issue_id = row[0]
            seq = row[1]
            date = row[2]
            commenter = row[3]
            comment = row[4]
            
            result = sample_analyze_sentiment(comment)
            
            score,magnitude = (result)
            print (score,magnitude)
        
        
    # Opening csv result file in append mode
            with open("/Users/amir/Documents/jsd_santiment_result.csv", "a+") as csv_save:
                writer = csv.writer(csv_save)
                if not header_added:
                    writer.writerow(['issue_id','seq','date','commenter','comment','score','magnitude'])
                    header_added = True
                writer.writerow([issue_id, seq, date,commenter,comment,score,magnitude])
                csv_save.close()


def sample_analyze_sentiment(text_content):

    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_sentiment(document, encoding_type=encoding_type) 
    
    score = format(response.document_sentiment.score)
    magnitude = format(response.document_sentiment.magnitude)
    
    return score, magnitude

    
if __name__ == '__main__':
    print(read_csv('/Users/amir/Documents/final clean data.csv'))

In [ ]:
#CSV Clean up

import csv
from bs4 import BeautifulSoup
from markdown import markdown
import re
header_added = False

def read_csv(file_name):
    header_added = False
    sep = '----'
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        next(reader)
        
    # Reading row by row,removing markdown,special character and spaces
        for row in reader:
            if not re.search('(?:\@)',row[3]):
                continue
            if re.search('(?:\@servicerocket.com)',row[3]):
                continue
            issue_id = row[0]
            seq = row[1]
            date = row[2]
            commenter = row[3]
            comment = row[4]
            rest = comment.split(sep, 1)[0]
            markdown = markdown_to_text(rest)
            sentence = ' '.join(markdown.split())
            for k in sentence.split("\n"):
                final = " ".join(re.findall(r"[a-zA-Z0-9]+", k))
            
    # Opening csv result file in append mode
            with open("/Users/amir/Documents/clean_data.csv", "a+") as csv_save:
                writer = csv.writer(csv_save)
                if not header_added:
                    writer.writerow(['issue_id','seq','date','commenter','comment','clean_comment'])
                    header_added = True
                writer.writerow([issue_id,seq,date,commenter,comment, final])
                csv_save.close()
            
def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

if __name__ == '__main__':
    print(read_csv('/Users/amir/Documents/comment.csv'))